In [ ]:
pip install pybaseball

In [ ]:
# 2023 season injured player list

import requests
from bs4 import BeautifulSoup

url = 'https://www.spotrac.com/mlb/injured-list/2023/cumulative-player/pitching/'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
tbody = soup.find_all('tbody')[3]
players = tbody.find_all(class_='player')
player_list = []
player_lists = []
player_url = []
player_dic_list = []

for i in range(len(players) - 1):
  player_list.append(players[i].find('a').text)
  player_url.append(players[i].find('a').get('href'))
  player_dic_list.append({'player':players[i].find('a').text, 'year': 2023,'Shoulder':0, 'Arm':0, 'Elbow':0, 'ribs':0,'oblique':0,'Back':0,'Neck':0,'Knee':0,'Finger':0,'Ankle':0,'Hip':0,'Hamstring':0,'Achilles':0,'Wrist':0,'Calf':0,'Biceps':0,'Hand':0,'Leg':0,'Shin':0})
player_lists.append(player_list)

In [ ]:
# 2022 season pitcher data

from pybaseball import pitching_stats_range
import pandas as pd
import numpy as np

pitch_data_2022 = pitching_stats_range('2022-04-01', '2022-10-30')

In [ ]:
# 2023 player injury history

histories = []
pitch_data_list = [pitch_data_2022]
for pitch_data, player_list in zip(pitch_data_list, player_lists):
  all_pitcher_name_list = pitch_data['Name'].values.tolist()
  all_pitcher_team_list = pitch_data['Tm'].values.tolist()

  #converting name
  for i, name in enumerate(all_pitcher_name_list):
    name = name.replace(' ', '-')
    all_pitcher_name_list[i] = name

  for name, team in zip(all_pitcher_name_list, all_pitcher_team_list):
    player_url = f'https://www.spotrac.com/mlb/{team}/{name}/injuries/'
    try:
      res = requests.get(player_url)
    except:
      continue
    soup = BeautifulSoup(res.text, 'html.parser')
    try:
        table = soup.find_all(class_='salaryTable hidden-xs')[0]
    except IndexError:
      continue


    tbody = table.find_all('tbody')[0]
    tr = tbody.find_all('tr')
    injuries = []
    for i in range(len(tr)):
      year = tr[i].find_all('td')[0].text
      td = tr[i].find_all('td')[2].text
      injury_dic = {'name': name.replace('-', ' '), 'year':year[-4:], 'injury':td}
      injuries.append(injury_dic)

    histories.append(injuries)

In [ ]:
#counting injuries

for i in range(len(player_dic_list)):
  for j in range(len(histories[i])):
    if int(histories[i][j]['year']) <= int(player_dic_list[i]['year']):
      if histories[i][j]['injury'] in player_dic_list[i]:
        injury_part = histories[i][j]['injury']
        player_dic_list[i][injury_part] = player_dic_list[i][injury_part] + 1

In [ ]:
# database creation

count = 0
year = 2021
pitch_data_list = [pitch_data_2022]
for pitch_data, player_list in zip(pitch_data_list, player_lists):
  year += 1
  pitch_data = pitch_data[['Name','Age', 'Lev', 'Tm', 'G', 'GS', 'W', 'L', 'SV', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'ERA', 'AB', '2B', '3B', 'IBB', 'GDP', 'SF', 'SB', 'CS', 'PO', 'BF', 'Pit', 'Str', 'StL', 'StS', 'GB/FB', 'LD', 'PU', 'WHIP', 'BAbip', 'SO9', 'SO/W']]
  # adding injury column
  pitch_data['Injury'] = 0
  # adding injury parts
  pitch_data['Shoulder'] = 0
  pitch_data['Arm'] = 0
  pitch_data['Elbow'] = 0
  pitch_data['Ribs'] = 0
  pitch_data['Oblique'] = 0
  pitch_data['Back'] = 0
  pitch_data['Neck'] = 0
  pitch_data['Knee'] = 0
  pitch_data['Finger'] = 0
  pitch_data['Ankle'] = 0
  pitch_data['Hip'] = 0
  pitch_data['Hamstring'] = 0
  pitch_data['Achilles'] = 0
  pitch_data['Wrist'] = 0
  pitch_data['Calf'] = 0
  pitch_data['Biceps'] = 0
  pitch_data['Hand'] = 0
  pitch_data['Leg'] = 0
  pitch_data['Shin'] = 0
  pitch_data['Year'] = year

  player_name_df = pitch_data['Name']
  players_name_list = player_name_df.tolist()

  for name in players_name_list:
    for i, history in enumerate(histories):
      if history != []:
        if history[0]['name'] == name:
          for i in range(len(history)):
            if ((int(history[i]['year']) <= year) and (int(history[i]['year']) != 1)) and (history[i]['injury'] in ['Shoulder', 'Arm', 'Elbow', 'Ribs','Oblique','Back','Neck','Knee','Finger','Ankle','Hip','Hamstring','Achilles','Wrist','Calf','Biceps','Hand','Leg','Shin']):
              pitch_data.loc[(pitch_data['Name'] == history[0]['name']) & (pitch_data['Year'] == year), history[i]['injury']] += 1
          break


  # if a player is injured, injury = 1
  for player in player_list:
    pitch_data.loc[pitch_data['Name'] == player, 'Injury'] = 1

  # replacing missing values&inf to 0
  pitch_data = pitch_data.replace([np.inf, -np.inf], np.nan)
  pitch_data = pitch_data.fillna(0)
  #conbining DataFrames
  count = count + 1
  if count == 1:
    pitch_data_all = pitch_data
  else:
    pitch_data_all = pd.concat([pitch_data_all, pitch_data])

# creating csv file
pitch_data_all.to_csv('pitch_data_2023.csv')
# downloading csv file
from google.colab import files
files.download('pitch_data_2023.csv')